In [16]:
import http.client
import json

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "3c99262600msh5da2d3d13033579p1c677ejsnc94020f9ff0e",

    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

# search

# get top transfers
conn.request("GET", "/football-get-top-transfers", headers=headers)

# # get top market values
# conn.request("GET", "/football-get-market-value-transfers?page=1", headers=headers)

# # get top players by assists?  --> leagueId --> + league BD
# conn.request("GET", "/football-get-top-players-by-assists?leagueid=47", headers=headers)

# # by goals
# conn.request("GET", "/football-get-top-players-by-goals?leagueid=47", headers=headers)

# # by rating
# conn.request("GET", "/football-get-top-players-by-rating?leagueid=47", headers=headers)

# # get popular league
# conn.request("GET", "/football-popular-leagues", headers=headers)



res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","response":{"transfers":[{"name":"Kevin Denkey","playerId":820477,"position":{"label":"ST","key":"striker_short"},"transferDate":"2024-11-22T01:17:59Z","transferText":[null,null,null],"fromClub":"Cercle Brugge","fromClubId":9984,"toClub":"Cincinnati","toClubId":722265,"fee":{"feeText":"fee","localizedFeeText":"transfer_fee","value":15300000},"transferType":{"text":"contract","localizationKey":"contract"},"contractExtension":false,"onLoan":false,"fromDate":"2025-01-01T00:00:00Z","toDate":"2028-12-31T00:00:00Z","marketValue":16000000},{"name":"Paul Pogba","playerId":248453,"position":{"label":"AM","key":"centerattackingmidfielder_short"},"transferDate":"2024-11-15T20:08:04Z","transferText":[null,null,null],"fromClub":"Juventus","fromClubId":9885,"toClub":"Free agent","toClubId":2,"fee":null,"transferType":{"text":"contract","localizationKey":"contract"},"contractExtension":false,"onLoan":false,"fromDate":"2024-11-30T00:00:00Z","marketValue":16000000},{"name":"Raphaël 

In [17]:
# Extract transfers data
conn.request("GET", "/football-get-top-transfers?page=1", headers=headers)
res = conn.getresponse()
data = res.read()
api_response = json.loads(data.decode("utf-8")) 

transfers = api_response["response"]["transfers"]

# Generate SQL INSERT statements
insert_statements = []
for transfer in transfers:
    name = transfer["name"].replace("'", "''")  # Escape single quotes
    player_id = transfer["playerId"]
    position = transfer["position"]["label"]
    transfer_date = transfer["transferDate"].replace("T", " ").replace("Z", "")
    from_club = transfer["fromClub"].replace("'", "''")
    from_club_id = transfer["fromClubId"]
    to_club = transfer["toClub"].replace("'", "''")
    to_club_id = transfer["toClubId"]
    
    # Handle 'fee' key and its 'value' safely
    fee = transfer.get("fee")  # Get the 'fee' dictionary or None if it doesn't exist
    fee_value = fee["value"] if fee and "value" in fee else "NULL"  # Safely get 'value' or default to "NULL"
    
    transfer_type = transfer["transferType"]["text"]
    contract_extension = "TRUE" if transfer["contractExtension"] else "FALSE"
    on_loan = "TRUE" if transfer["onLoan"] else "FALSE"
    market_value = transfer["marketValue"]

    insert_statements.append(
        f"INSERT INTO football_transfers (name, player_id, position, transfer_date, "
        f"from_club, from_club_id, to_club, to_club_id, fee_value, transfer_type, "
        f"contract_extension, on_loan, market_value) VALUES ('{name}', {player_id}, '{position}', "
        f"'{transfer_date}', '{from_club}', {from_club_id}, '{to_club}', {to_club_id}, "
        f"{fee_value}, '{transfer_type}', {contract_extension}, {on_loan}, {market_value});"
    )
# Write to 2-data.sql
sql_file_path = "../services/recommender-db/sql/2-data.sql"
with open(sql_file_path, "w", encoding="utf-8") as file:  # Specify UTF-8 encoding
    file.write("-- Generated SQL insert statements for football transfers\n")
    file.write("\n".join(insert_statements))

print(f"2-data.sql generated successfully at {sql_file_path}")


2-data.sql generated successfully at ../services/recommender-db/sql/2-data.sql


In [18]:
import http.client
import json

# Constants for the API
BASE_URL = "free-api-live-football-data.p.rapidapi.com"
HEADERS = {
    'x-rapidapi-key': "3c99262600msh5da2d3d13033579p1c677ejsnc94020f9ff0e",
    'x-rapidapi-host': BASE_URL
}

# Initialize HTTP connection
conn = http.client.HTTPSConnection(BASE_URL)



In [19]:
def fetch_data(endpoint, params=None):
    if params:
        endpoint = f"{endpoint}?{params}"
    conn.request("GET", endpoint, headers=HEADERS)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))


def process_records(records, table_name, schema):
    insert_statements = []
    for record in records:
        values = []
        for field_name, default_value, is_string in schema:
            value = record.get(field_name, default_value)
            if is_string and value is not None:
                value = value.replace("'", "''")  # Escape single quotes
                value = f"'{value}'"
            elif value is None:
                value = "NULL"
            values.append(value)
        values_str = ", ".join(map(str, values))
        insert_statements.append(f"INSERT INTO {table_name} VALUES ({values_str});")
    return insert_statements

def write_to_sql_file(file_path, insert_statements, section_title=None):
    with open(file_path, "a", encoding="utf-8") as file:
        if section_title:
            file.write(f"-- {section_title}\n")
        file.write("\n".join(insert_statements) + "\n")


In [20]:
transfers_schema = [
    ("name", "NULL", True),
    ("playerId", "NULL", False),
    ("position.label", "NULL", True),  # Nested key
    ("transferDate", "NULL", True),
    ("fromClub", "NULL", True),
    ("fromClubId", "NULL", False),
    ("toClub", "NULL", True),
    ("toClubId", "NULL", False),
    ("fee.value", "NULL", False),  # Nested key
    ("transferType.text", "NULL", True),
    ("contractExtension", False, False),
    ("onLoan", False, False),
    ("marketValue", "NULL", False),
]

transfers_data = fetch_data("/football-get-top-transfers?page=1")
transfers = transfers_data["response"]["transfers"]
transfer_statements = process_records(transfers, "football_transfers", transfers_schema)
write_to_sql_file(sql_file_path, transfer_statements, "Top Transfers")